In [1]:
import gym
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle

#for text processing
import spacy
import re
import pandas as pd
env = gym.make("Taxi-v2").env

env.render()

DeprecatedEnv: Env Taxi-v2 not found (valid versions include ['Taxi-v3'])

In [2]:
import gym
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle

#for text processing
import spacy
import re
import pandas as pd
env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



#### There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."

### Fetching Origing, Destination, and Time of Pickup from the sms data 

In [3]:
def fetch_pickup_drop(text):
    
    time = re.findall(r'[\d+]+ PM',text)
    if len(time):
        pickup_time = time[0].strip()
    else:
        time = re.findall(r'[\d+]+ AM',text)
        pickup_time = time[0].strip()

    
    city = pd.read_csv("./city.csv")
    all_cities = list(city['location'].values)
    
    citynames_in_sms = []
    for city in all_cities:
        if city in line:
            citynames_in_sms.append(city)
            
    drop_city = ""
    pickup_city = ""
    if len(citynames_in_sms) == 2:
        for city in citynames_in_sms:
            res2 = re.findall('to '+city,line)
            if len(res2):
                print("")
            else:
                res2 = re.findall('destination '+city,line)
            if len(res2):
                print("")
            else:
                res2 = re.findall('for '+city,line)
            if len(res2):
                drop_city = city
                citynames_in_sms.remove(city)
                pickup = citynames_in_sms[0]
                break;           
    return [pickup, drop_city, pickup_time]

In [4]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


## Summing up the Q-Learning Process
Breaking it down into steps, we get

Initialize the Q-table by all zeros.

Start exploring actions: 

For each state, select any one among all possible actions for the current state (S).

Travel to the next state (S') as a result of that action (a).

For all possible actions from the state (S') select the one with the highest Q-value.

Update Q-table values using the equation.

Set the next state as the current state.

If goal state is reached, then end and repeat the process.


## Exploiting learned values
After enough random exploration of actions, the Q-values tend to converge serving our agent as an action-value function which it can exploit to pick the most optimal action from a given state.

There's a tradeoff between exploration (choosing a random action) and exploitation (choosing actions based on already learned Q-values). We want to prevent the action from always taking the same route, and possibly overfitting, so we'll be introducing another parameter called ϵ "epsilon" to cater to this during training.

Instead of just selecting the best learned Q-value action, we'll sometimes favor exploring the action space further. Lower epsilon value results in episodes with more penalties (on average) which is obvious because we are exploring and making random decisions.

In [5]:
#Initialize Q_table
import numpy as np
#write your code here

q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [6]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 1000001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")
np.save("./q_table.npy", q_table)

Episode: 1000000
Training finished.

Wall time: 30min


In [7]:
#Load trained q_table for evaluation

q_table = np.load("./q_table.npy")

In [8]:
def create_loc_dict(city_df):
    
    ## Create dictionary example, loc_dict['dwarka sector 23'] = 0
    
    loc_dict = {'dwarka sector 23': 0,'dwarka sector 21':1,'hauz khaas':2,'airport':3}   
    return loc_dict

In [9]:
def check_pick_up_drop_correction(pick_up, drop, line_num):
    #write your code here
    
    df = pd.read_csv("org_df.csv")
    original_origin = df['origin'][line_num]
    original_destination = df['dest'][line_num]
    if original_origin == pick_up and original_destination == drop:
        return True
    else:
        return False
    

In [10]:
"""Evaluate agent's performance after Q-learning"""

# 1) We need to take text drom "sms.txt" and fetch pickup and drop from it.
# 2) Generate the random state from an enviroment and change the pick up and drop as the fetched one
# 3) Evaluate you q_table performance on all the texts given in sms.txt.
# 4) Have a check if the fetched pickup, drop is not matching with original pickup, drop using orig.csv
# 5) If fetched pickup or/and drop does not match with the original, add penality and reward -10
# 6) Calculate the Total reward, penalities, Wrong pickup/drop predicted and Average time steps per episode.

total_epochs, total_penalties, total_reward, wrong_predictions = 0, 0, 0, 0


count = 0
time_list = []
f = open("./sms.txt", "r")
num_of_lines = 1000
city = pd.read_csv("./city.csv")

loc_dict = create_loc_dict(city)
# print(loc_dict)
line_num = 0
for line in f:
#     print(line)
    origin, destination, time_of_pickup = fetch_pickup_drop(line)
#     print(origin, destination, time_of_pickup)
    if not check_pick_up_drop_correction(origin, destination, line_num):
        total_reward = total_reward - 10
        total_penalties = total_penalties + 10
        wrong_predictions += 1
    
    pickup_int = loc_dict[origin]
    drop_int = loc_dict[destination]
    
    current_state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    out = env.decode(current_state)
    
#     new_state = env.encode(out, pickup_int, drop_int) 
    
    done = False
    
    while not done:
        action = np.argmax(q_table[current_state])
        current_state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1
            

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs
    total_reward += reward 
        
 ##Write your code here

    line_num += 1


print(f"Results after {num_of_lines} episodes:")
print(f"Average timesteps per episode: {total_epochs / num_of_lines}")
print(f"Average penalties per episode: {total_penalties / num_of_lines}")
print(f"Total number of wrong predictions", wrong_predictions)
print("Total Reward is", total_reward)

In [11]:
print(f"Results after {num_of_lines} episodes:")
print(f"Average timesteps per episode: {total_epochs / num_of_lines}")
print(f"Average penalties per episode: {total_penalties / num_of_lines}")
print(f"Total number of wrong predictions", wrong_predictions)
print("Total Reward is", total_reward)

Results after 1000 episodes:
Average timesteps per episode: 13.075
Average penalties per episode: 0.0
Total number of wrong predictions 0
Total Reward is 20000
